### Etapa 1 - Processo de Segmentação (Remoção de Fundo Complexo)

In [1]:
# Bibliotecas Utilizadas
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from sklearn.cluster import KMeans
import skimage.color
import cv2
from scipy import misc
from skimage.color import rgb2gray
from skimage import color
import skimage.metrics
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio
from PIL import Image
from scipy import ndimage
import functions as fun
import time
import pandas as pd
import statistics as st
import glob
from IPython.display import display, Javascript 
from os import chdir, getcwd, listdir
from os.path import isfile
import os
import oci
import boto3
from docx import Document
from docx.shared import Inches

#### Configurações e Funcionalidades Buckets - Oracle Cloud

In [163]:
# Função para configurações da Oracle Cloud
def config_cloud(flag):

    # Variável para URL de definição do Endpoint
    if flag == 'p':
        # Ambiente de Produção - Configuração Cloud
        url = "https://grinqewrovfi.compat.objectstorage.sa-saopaulo-1.oraclecloud.com" 
    
    elif flag =='t':
        # Ambiente de Testes - Configuração Cloud
        url = "https://grwbzp0j0zza.compat.objectstorage.sa-saopaulo-1.oraclecloud.com"
    
    # Chamada de função para configuração Cloud
    fun.configuration_Cloud()

    # Chamada de função para obter o "resource"
    resource_value = fun.resource()
       
    return url, resource_value

#### Obter as imagens do Banco de Imagens Embrapa

In [164]:
# Function to get objects selected bucket
def objects_to_bucket(flag, bucket, resource_value, url):
    # Path in defined
    path_in = "/home/datascience/db_imagens/"
    
    if contador_imagens ==0:
        os.makedirs(path_in)
        
    # Print all buckets
    try:
        for bucket_name in resource_value.buckets.all():
            print (bucket_name.name)
        print("\n Conexao com sucesso na cloud!!\n")
    except:
        print("Falha de conexao na cloud")    
    
    files = fun.getObjectsToBucket(str(bucket), resource_value)

    # Flag 'd' - Data download process - Boto3 (API)
    if flag == 'd':
        # Function call for download file: destination data science workspace - folder db_imagens
        fun.download_to_bucket(str(bucket), url, files, path_in)
        message = "Download de arquivos com sucesso!!"
    
    # Flag 's' - Data synchronism process - rclone (API)
    elif flag == 's':
        try:
            cmd = 'rclone sync ' + str(bucket)+ ":" + str(bucket) + " " + path_in
            os.system(cmd)
            message = "Arquivos sincronizados com successo!!"
        except:
            message = "Erro de sincronismo"
    
    return message, path_in

In [165]:
# Função para armazenar Filepaths para processamento
def armazenar_filepaths_processamento(contador_imagens):
    global nome_imagem
    # Configurar parâmetros dos diretórios
    path_in = 'db_imagens/'
          
    filepaths = [f for f in os.listdir(path_in) if f.endswith('.jpg')]
    number_files = len(filepaths)

    nome_imagem = str(filepaths[contador_imagens])
    pasta = nome_imagem[-12::]
    nomeTeste= 'Processamento '+ str(contador_imagens) + ' - Qualificação'
    
    return path_in, nome_imagem, pasta, filepaths, nomeTeste, number_files

#### Uso das Imagens do banco de dados da Embrapa
#### Parâmetros de Configuração do Processo

In [166]:
# Função para configuração de Parâmetros e Diretórios
def parameters_config(path_in, nome_imagem, coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4):
    
    path_out = 'Segmentacao_Etapa_1/' + nome_imagem + '/'
    path_out_labels = 'Segmentacao_Etapa_1/' + nome_imagem + '/' + 'labels/'
    os.makedirs(path_out)
    os.makedirs(path_out_labels)

    # Leitura do arquivo de Imagem no disco
    nome_imagem = nome_imagem    

    img_orig = plt.imread(path_in + nome_imagem)

    return path_out, img_orig, nome_imagem, path_out_labels

#### Equalização do Histograma da Imagem - Canal Verde

In [167]:
# Função para equalização de histograma da imagem
def equalizar_imagem(img_green_):
    img_green = cv2.equalizeHist(img_green_)

    #Plotar as Imagens
    plt.figure(figsize=[15,15])
    plt.subplot(121), plt.imshow(img_green_)
    plt.title('Imagem Canal Verde')
    plt.subplot(122), plt.imshow(img_green)
    plt.title('Imagem Canal Verde - Equalizado')
    plt.show()

    return img_green

#### Salvar a imagem do Canal Verde em disco

In [168]:
# Função para gravar em Disco a imagem Canal Verde
def salvar_imagem_verde(img_green_, img_green, url, path_out, nome_imagem):
    fun.salvarImagemSegmentadaDisco (img_green_, path_out + 'img_canal_verde_' + nome_imagem)
    fun.salvarImagemSegmentadaDisco (img_green, path_out + 'img_canal_verde_equal_' + nome_imagem)

    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'img_canal_verde_' + nome_imagem)
    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'img_canal_verde_equal_' + nome_imagem)

#### Processo de Limiarização da Imagem

In [169]:
## Função para limiarização da imagem (Canal Verde)
def limiarizar_imagem(img_green, th1, th2, url, path_out, nome_imagem):
    # Usar uma cópia da imagem do canal verde (para não alterar a variável original)
    img_gravada_verde = img_green.copy()

    # Chamada do método de Limiarização
    img_limiarizada = fun.threshold(img_gravada_verde, th1, th2)

    # Visualização da imagem
    plt.figure(figsize=[15, 15])
    plt.title('Imagem Limiarizada')
    plt.imshow(img_limiarizada)    

    # Chamada de função para gravar em Disco
    fun.salvarImagemSegmentadaDisco(img_limiarizada, path_out + 'img_limiarizada_' + nome_imagem)

    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'img_limiarizada_' + nome_imagem)

    return img_gravada_verde, img_limiarizada

In [170]:
# Função para binarizar imagem
def binarizar_imagem(img_limiarizada, th1, th2, url, path_out, nome_imagem):
    img_binar = fun.binarizarImagem(img_limiarizada, th1, th2)
    plt.title('Imagem Binarizada')
    plt.imshow(img_binar)

    # Chamada de função para gravar em Disco
    fun.salvarImagemSegmentadaDisco(img_binar, path_out + 'img_binarizada_' + nome_imagem)

    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-1', url,  path_out + 'img_binarizada_' + nome_imagem)

    return img_binar

In [171]:
# Função para recuperar os pixes de referência
def recuperar_pixels_ref(img_binar, img_orig, path_out, url, nome_imagem):

    #Padronização de nome de variável. Criação de cópia da imagem
    img = img_binar.copy()

    matriz_reconst_bin = fun.reconstruirMatrizPixelsReferencia(img_orig,img, 'IMG_SEGMENTADA')
    plt.title('Imagem Segmentada - Pixels Recuperados')
    plt.imshow(matriz_reconst_bin)

    # Chamada de função para gravar em Disco imagem reconstruida na etapa de limiarização
    #fun.salvarImagemSegmentadaDisco (matriz_reconst_bin, str(fun.path) + 'verdes\imagem_reconstruida_etapa_lim.jpg')
    fun.salvarImagemDiscoRGB(matriz_reconst_bin, path_out + 'imagem_reconstruida_etapa_lim_' + nome_imagem)

    # Function call Upload files function from workspace files to Selected Bucket
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'imagem_reconstruida_etapa_lim_' + nome_imagem)

    # Padronização da imagem reconstruída
    img = matriz_reconst_bin.copy()

    return img

#### Método Elbow - Técnica usada para a obtenção do valor ideal para 'K'- Algoritmo K-Means

In [172]:
# Chamada da Técnica Elbow (Estimativa para número de clusters - K-Means)
#fun.elbow (img)

#### Aplicação do Método de Segmentação - Uso do K-means no espaço de cores RGB

In [173]:
# Função algoritmo K-Means
def algoritmo_Kmeans(img):
    num_clusters = 6
    km,labels,img_labels = fun.kmeans(num_clusters,img)

    # Exibir a imagem com todos os labels juntos
    plt.figure(figsize=[15,15])
    plt.title('Todos os Labels K-Means Juntos')
    plt.imshow(img_labels)
    
    return km,labels,img_labels, num_clusters

#### Visualização das imagens original e os labels da Segmentação (mais significativos)

In [174]:
# Função para exibição dos Labels Gerados pelo Algoritmo K-Means
def exibir_labels_kmeans(img_labels_closed):
    plt.figure(figsize=[15, 15])
    plt.subplot(1, 6, 1)
    plt.imshow(img_labels_closed[0],'gray')
    plt.title('Label 0')

    plt.subplot(1, 6, 2)
    plt.imshow(img_labels_closed[1],'gray')
    plt.title('Label 1')

    plt.subplot(1, 6, 3)
    plt.imshow(img_labels_closed[2],'gray')
    plt.title('Label 2')

    plt.subplot(1, 6, 4)
    plt.imshow(img_labels_closed[3],'gray')
    plt.title('Label 3')

    plt.subplot(1, 6, 5)
    plt.imshow(img_labels_closed[4],'gray')
    plt.title('Label 4')

    plt.subplot(1, 6, 6)
    plt.imshow(img_labels_closed[5],'gray')
    plt.title('Label 5')    

#### Gravar imagens: processo restauração de pixels
#### (Pixels do Label (K-Means) Significativo x Pixels imagem Original)

In [175]:
# Função para restaurar os pixels originais da imagem de rederência - Etapa Final Sementação
def restaurar_pixels_kmeans(img_orig,img_labels_closed, coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4):
    matriz_restaurada =[]
    # Restauração de label 0
    matriz_restaurada_0 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[0], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_0)
    # Restauração de label 1
    matriz_restaurada_1 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[1], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_1)
    # Restauração de label 2
    matriz_restaurada_2 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[2], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_2)
    # Restauração de label 3
    matriz_restaurada_3 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[3], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_3)
    # Restauração de label 4
    matriz_restaurada_4 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[4], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_4)
    # Restauração de label 5
    matriz_restaurada_5 = fun.reconstruirMatrizPixelsReferencia(img_orig,img_labels_closed[5], 'IMG_FINAL',\
                                                                coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4)
    matriz_restaurada.append(matriz_restaurada_5)

    return matriz_restaurada

In [176]:
# Função para exibir as imagens Restauradas - Labels de 0 até 2
def exibir_labels_0_2(matriz_restaurada):
    plt.figure(figsize=[20, 20])
    plt.subplot(1, 3, 1)
    plt.imshow(matriz_restaurada[0])
    plt.title('Segmentada - Label 0')
    plt.subplot(1, 3, 2)
    plt.imshow(matriz_restaurada[1])
    plt.title('Segmentada - Label 1')
    plt.subplot(1, 3, 3)
    plt.imshow(matriz_restaurada[2])
    plt.title('Segmentada - Label 2')

In [177]:
# Função para exibir as imagens Restauradas - Labels de 3 até 5
def exibir_labels_3_5(matriz_restaurada):
    plt.figure(figsize=[20, 20])
    plt.subplot(1, 3, 1)
    plt.imshow(matriz_restaurada[3])
    plt.title('Segmentada - Label 3')
    plt.subplot(1, 3, 2)
    plt.imshow(matriz_restaurada[4])
    plt.title('Segmentada - Label 4')
    plt.subplot(1, 3, 3)
    plt.imshow(matriz_restaurada[5])
    plt.title('Segmentada - Label 5')

In [178]:
# Função para salvar as imagens Restauradas - Todos os Labels
def salvar_img_restauradas(matriz_restaurada, path_out_labels, url, nome_imagem):
    # Chamada de função para upload de arquivos da 'area de trabalho' para o bucket selecionado
    #  para todos os labels do k-means
    fun.salvarImagemDiscoRGB(matriz_restaurada[0], path_out_labels + 'img_segmentada_restaurada_0_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_0_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[1], path_out_labels + 'img_segmentada_restaurada_1_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_1_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[2], path_out_labels + 'img_segmentada_restaurada_2_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_2_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[3], path_out_labels + 'img_segmentada_restaurada_3_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_3_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[4], path_out_labels + 'img_segmentada_restaurada_4_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_4_' + nome_imagem)
    fun.salvarImagemDiscoRGB(matriz_restaurada[5], path_out_labels + 'img_segmentada_restaurada_5_' + nome_imagem)
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels + 'img_segmentada_restaurada_5_' + nome_imagem)

#### Exibe label escolhido e da imagem segmentada correspondente

In [179]:
# Função para Exibição do label e imagem escolhidos
def exibir_label_imagem_escolhidos(img_labels_closed, url, label_significativo, path_out_labels, nome_imagem):
    img_gravada = img_labels_closed[label_significativo]

    matriz_escolhida = cv2.cvtColor(cv2.imread(path_out_labels + 'img_segmentada_restaurada_' + \
                                               str(label_significativo) + '_' + nome_imagem), cv2.COLOR_BGR2RGB)

    # Plotar as imagens
    plt.figure(figsize=[15, 15])
    plt.subplot(1, 2, 1)
    plt.imshow(img_gravada)
    plt.title('Label Escolhido')
    plt.subplot(1, 2, 2)
    plt.imshow(matriz_escolhida)
    plt.title('Imagem Segmentada Correspondente')
    
    # Gravar arquivo em disco - Resultado
    plt.savefig(path_out_labels + 'label_escolhido_img_segmentada.png', format='png')
    fun.salvarImagemDiscoRGB(matriz_escolhida, path_out_labels +\
                             '/img_segment_escolh_label_' + str(label_significativo) + '_' + nome_imagem)
    
    # Chamada de função para upload de arquivos da 'area de trabalho' para o bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out_labels +\
                             '/img_segment_escolh_label_' + str(label_significativo) + '_' + nome_imagem)

    return matriz_escolhida, img_gravada

#### Pré-Processamento: Suavização (Filtro de Mediana)

In [180]:
# Função para aplicação do Filtro de Mediana
def filtro_mediana(img_orig, path_out, url, nome_imagem):
    img_suave = cv2.medianBlur(img_orig,3)

    #Plotar as Imagens
    plt.figure(figsize=[20,20])
    plt.subplot(121),plt.imshow(img_orig)
    plt.title('Imagem Original')
    plt.subplot(122)
    plt.imshow(img_suave)
    plt.title('Suavização - Filtro de Mediana')
    plt.show()

    # Chamada de função para gravar em Disco a imagem suavizada
    fun.salvarImagemSegmentadaDisco (img_suave, path_out + 'img_filtro_mediana_' + nome_imagem)
    # Chamada de função para upload de arquivos da 'area de trabalho' para o bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'img_filtro_mediana_' + nome_imagem)

    return img_suave

#### Métricas Processo de Filtragem (Filtro Mediana)

In [181]:
# Função para calcular métricas processo filtragem
def metricas_filtragem(img_orig, path_out, path_out_labels, url, img_suave):
    
    img_gray_1 = cv2.cvtColor(cv2.imread(path_in + nome_imagem), cv2.COLOR_BGR2RGB)
    img_gray_2 = cv2.cvtColor(cv2.imread(path_out_labels + 'img_segmentada_restaurada_' + str(label_significativo) + '_' + nome_imagem), cv2.COLOR_BGR2RGB)
    img_gray_3 = cv2.cvtColor(cv2.imread(path_out + 'img_filtro_mediana_' + nome_imagem), cv2.COLOR_BGR2RGB)
    
    text = ''
    text+= "----------------------------------\n"
    text+= "Etapa 1 - " + nome_imagem + '\n'
    # Redimensiona o array para 2 dimensões
    data_1 = img_orig.reshape(img_orig.shape[0]*img_orig.shape[1], 3)
    data_2 = img_suave.reshape(img_suave.shape[0]*img_suave.shape[1], 3)

    # Calcular MSE (Mean Squared Error)
    # Imagem Original e Imagem com Filtro de Mediana
    mse = mean_squared_error(data_1, data_2)
    text+="MSE: " + str(mse) + '\n'

    # Peak signal to noise ratio
    # Imagem Original e Imagem com Filtro de Mediana
    psnr = peak_signal_noise_ratio(data_1, data_2)
    text+="PSNR: " + str(psnr) + '\n'

    ## Calcular o Structural Similarity Index (SSIM) entre duas imagens
    # Retorna a diferença entre as imagens
    (score, diff) = structural_similarity(img_gray_1, img_gray_3, full=True, multichannel=True)
    diff = (diff * 255).astype("uint8")
    # Imprimir os valores de Score e diff
    text +="SSIM: {}".format(score)
    text+= "\n----------------------------------"

    # Gravar arquivo de texto com as métricas sementes_calculadas_janela
    with open(path_out + 'metricas_filtragem_' + nome_imagem + '.txt', 'w') as f:
        f.write(text)
        
    # Chamada de função para upload de arquivos da 'area de trabalho' para o bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out +\
                           'metricas_filtragem_' + nome_imagem + '.txt')
       
    return mse, psnr, score, diff, text, img_gray_1, img_gray_2, img_gray_3

#### Comparação de Histogramas - Imagem Original e Segmentada

In [182]:
# Função para comparar histogramas das imagens
def comparar_histogramas(url, path_out, img_gray_1, img_gray_2, coordJanela_1,coord_Janela_2,\
                         coordJanela_3,coordJanela_4):
    # Considerada a janela (Objeto de Interesse)
    # Criar a Máscara
    mask = np.zeros(img_gray_1.shape[:2], np.uint8)
    mask[coordJanela_1:coord_Janela_2, coordJanela_3:coordJanela_4] = 255
    masked_img = cv2.bitwise_and(img_gray_1,img_gray_1,mask = mask)

    # Calcular o Histograma com a Máscara
    hist_mask = cv2.calcHist([img_gray_1],[0],mask,[256],[0,255])
    hist_mask_2 = cv2.calcHist([img_gray_2],[0],mask,[256],[0,255])

    # Exibir o gráfico de Comparação dos Histogramas
    plt.figure(figsize=[15,15])
    plt.subplot(224), plt.plot(hist_mask, color='red', label='Original'),\
        plt.plot(hist_mask_2, color ='blue', label='Segmentada')
    plt.title('Histogramas_' + nome_imagem)
    plt.legend()
    plt.xlim([0,255])
  
    # Gravar arquivo em disco - Resultado Canal Verde
    plt.savefig(path_out + 'histogramas_' + nome_imagem, format='jpg')
    
    # Chamada de função para upload de arquivos da 'area de trabalho' para o bucket selecionado
    fun.upload_from_bucket('BK-segmented-images-1', url, path_out + 'histogramas_' + nome_imagem)

    plt.show() 
        
    return mask, masked_img, hist_mask, hist_mask_2

#### Inicialização de Variáveis - Execução do Processo de Segmentação Etapa 1

In [370]:
### ------------------------------------ ###
### Inicialização do contador de imagens
### ------------------------------------ ###
global contador_imagens
contador_imagens =29

### -------------------------------------------------------- ###
# Parâmetros das Coordenadas da Janela do Objeto de Interesse
### -------------------------------------------------------- ###
global coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4
#550, 2700, 1300, 2700 => Parâmetros para folhas verticais
#900, 2300, 900, 2900 => Parâmetros para folhas horizontais

#Considerados os parâmetros das folhas verticais e horizontais
#   para deixar a janela com a abertura para contemplar ambos os casos
#   550, 2700, 700, 3200

coordJanela_1, coordJanela_2, coordJanela_3, coordJanela_4 = 550, 2700, 700, 3200

#### Execução do Processo de Segmentação Etapa 1 (Parte 1)


In [ ]:
#####---------------------------------------#####
 ## PASSOS APÓS A EXECUÇÃO DA PRIMEIRA IMAGEM ##
#####---------------------------------------#####

# Chamada de função para configuração da cloud
url,resource_value = config_cloud('p')

# Chamada de função para busca de objetos na cloud mo bucket selecionado
message, path_in = objects_to_bucket('s', 'BK-images-1', resource_value, url)

# Chamada de função para armazenar Filepaths para processamento
path_in, nome_imagem, pasta, filepaths, nomeTeste,\
    number_files = armazenar_filepaths_processamento(contador_imagens)
print(str(filepaths) + '\n')

# Chamada de função para configuração de Parâmetros e Diretórios
path_out, img_orig, nome_imagem, path_out_labels = parameters_config(path_in, nome_imagem, coordJanela_1,\
                                                     coordJanela_2, coordJanela_3, coordJanela_4)

## Chamar função de exibição de propriedades da imagem
fun.caracteristicasImagem(img_orig)

# Chamada de função para plotar imagens com os canais de cores separados
img_red_, img_green_, img_blue_ = fun.plotaGraficoCanaisImagem(img_orig)

# Chamada de função para equalização de histograma da imagem
img_green = equalizar_imagem(img_green_)

# Chamada de função para gravar em Disco a imagem Canal Verde
salvar_imagem_verde(img_green_, img_green, url, path_out, nome_imagem)

## Chamada de função para limiarização da imagem (Canal Verde)
img_gravada_verde, img_limiarizada = limiarizar_imagem(img_green, 0, 200, url, path_out, nome_imagem)

# Chamada de função para binarizar imagem
img_binar = binarizar_imagem(img_limiarizada, 0, 200, url, path_out, nome_imagem)

# Chamada de função para recuperar os pixes de referência
img = recuperar_pixels_ref(img_binar, img_orig, path_out, url, nome_imagem)

# Chamada de função algoritmo K-Means
km,labels,img_labels, num_clusters = algoritmo_Kmeans(img)

# Chamada Função para exibir rotulos (baseado no número de clusters)
# Exibe os rótulos separados da imagem processada (Cada Cluster do K-Means gera um rótulo)
img_labels_closed = fun.exibeRotulos(num_clusters, img_labels, img)

# Chamada de função para exibir os Labels gerados pelo algoritmo K-Means
exibir_labels_kmeans(img_labels_closed)

#### Ação Supervisionada

In [ ]:
global label_significativo

####-------------------------------------------------------####
####----------------(AÇÃO SUPERVISIONADA)------------------####
# Escolha do label mais significativo do algoritmo K-Means
print("Imagem em Análise: " + nome_imagem + "\n")
label_significativo = int(input("Digite o valor do Label mais significativo (K-Means): "))
####-------------------------------------------------------####

#### Execução do Processo de Segmentação Etapa 1 (Parte 2)

In [ ]:
# Função para restaurar os pixels originais da imagem de rederência - Etapa Final Sementação
matriz_restaurada = restaurar_pixels_kmeans(img_orig, img_labels_closed, coordJanela_1,\
                                            coordJanela_2, coordJanela_3, coordJanela_4)

# Chamada de função para exibir as imagens Restauradas - Labels de 0 até 2
exibir_labels_0_2(matriz_restaurada)

# Chamada de função para exibir as imagens Restauradas - Labels de 3 até 5
exibir_labels_3_5(matriz_restaurada)

# Chamada de função para salvar as imagens Restauradas - Todos os Labels
salvar_img_restauradas(matriz_restaurada, path_out_labels, url, nome_imagem)

# Chamada de função para Exibição do label e imagem escolhidos
matriz_escolhida, img_gravada = exibir_label_imagem_escolhidos(img_labels_closed, url,\
                                                               label_significativo, path_out_labels, nome_imagem)    

# ------ Chamada de função para aplicação do Filtro de Mediana -------
img_suave = filtro_mediana(matriz_escolhida, path_out, url, nome_imagem)

# Chamada de função para calcular métricas processo filtragem
mse, psnr, score, diff, text, img_gray_1, img_gray_2,\
    img_gray_3 = metricas_filtragem(img_orig, path_out, path_out_labels, url, img_suave)

# Chamada de função para comparar histogramas das imagens
mask, masked_img, hist_mask, hist_mask_2 = comparar_histogramas(url, path_out, img_gray_1, img_gray_2,\
     coordJanela_1,coordJanela_2, coordJanela_3,coordJanela_4)

# Verifica se o processamento está finalizado
if contador_imagens == number_files:
    print("\nContador incrementado =>" + str(contador_imagens) + " ATENÇÃO!! ÚLTIMA IMAGEM PARA PROCESSAR!! ")
else:
    contador_imagens+=1
    print("\n\n Contador incrementado => Próxima Imagem à Processar: " + str(contador_imagens))